# 🔍 Анализ аналитики сайта Viktorija Autokool

## Цель анализа:
- Анализ поведения пользователей
- Пути навигации (funnel analysis)
- Время на странице
- Конверсия до checkout
- Визуализация данных
- Инсайты и рекомендации по улучшению UX

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import re

# Настройка отображения
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Библиотеки загружены")

## 📊 Загрузка и подготовка данных

In [ ]:
# Загрузка данных
df = pd.read_csv('Analytics/inspectlet-results-export.csv')
print(f"📈 Загружено {len(df)} записей")
df.head()

## 🧹 Очистка и подготовка данных

In [ ]:
# Функция для извлечения времени сессии в минутах
def extract_session_time(time_str):
    if pd.isna(time_str):
        return 0
    
    # Извлечение времени из строки
    time_parts = time_str.split()
    total_minutes = 0
    
    for i, part in enumerate(time_parts):
        if part.isdigit():
            if i + 1 < len(time_parts):
                next_part = time_parts[i + 1]
                if 'min' in next_part:
                    total_minutes += int(part)
                elif 'sec' in next_part:
                    total_minutes += int(part) / 60
    
    return total_minutes

# Функция для извлечения количества страниц
def extract_pages_count(pages_str):
    if pd.isna(pages_str):
        return 0
    
    # Извлечение числа страниц
    match = re.search(r'(\d+)\s+page', pages_str)
    if match:
        return int(match.group(1))
    return 0

# Применение функций очистки
df_clean = df.copy()
df_clean['session_time_minutes'] = df_clean['Session Length'].apply(extract_session_time)
df_clean['pages_count'] = df_clean['Pages Visited'].apply(extract_pages_count)

# Анализ путей навигации
df_clean['has_checkout'] = df_clean['Pages Visited'].str.contains('checkout', case=False, na=False)
df_clean['has_packages'] = df_clean['Pages Visited'].str.contains('packages', case=False, na=False)

print("✅ Данные очищены и подготовлены")
df_clean.head()

## 📊 Основные метрики и статистика

In [ ]:
# Основные метрики
print("🎯 ОСНОВНЫЕ МЕТРИКИ САЙТА")
print("=" * 50)

total_sessions = len(df_clean)
avg_session_time = df_clean['session_time_minutes'].mean()
avg_pages = df_clean['pages_count'].mean()
bounce_rate = (df_clean['pages_count'] == 1).mean() * 100
checkout_conversion = df_clean['has_checkout'].mean() * 100

print(f"📊 Общее количество сессий: {total_sessions}")
print(f"⏱️  Средняя длительность сессии: {avg_session_time:.2f} минут")
print(f"📄 Среднее количество страниц: {avg_pages:.2f}")
print(f"🔄 Bounce Rate: {bounce_rate:.1f}%")
print(f"💰 Конверсия до checkout: {checkout_conversion:.1f}%")

## 🎯 Анализ поведения пользователей

In [ ]:
# Сегментация пользователей по длительности сессии
print("👥 СЕГМЕНТАЦИЯ ПОЛЬЗОВАТЕЛЕЙ ПО ДЛИТЕЛЬНОСТИ СЕССИИ")
print("=" * 60)

# Создание сегментов
def categorize_session_time(minutes):
    if minutes < 1:
        return 'Быстрые (< 1 мин)'
    elif minutes < 5:
        return 'Короткие (1-5 мин)'
    elif minutes < 15:
        return 'Средние (5-15 мин)'
    else:
        return 'Долгие (> 15 мин)'

df_clean['session_category'] = df_clean['session_time_minutes'].apply(categorize_session_time)

# Анализ сегментов
session_segments = df_clean.groupby('session_category').agg({
    'Display Name': 'count',
    'session_time_minutes': ['mean', 'median'],
    'pages_count': ['mean', 'median'],
    'has_checkout': 'mean'
}).round(3)

print(session_segments)

## 🛤️ Пути навигации (Funnel Analysis)

In [ ]:
# Анализ воронки конверсии
print("🛤️  ВОРОНКА КОНВЕРСИИ")
print("=" * 50)

# Основные этапы воронки
funnel_steps = {
    'Главная страница': len(df_clean),
    'Услуги/Packages': df_clean['has_packages'].sum(),
    'Checkout': df_clean['has_checkout'].sum()
}

print("📊 Количество пользователей на каждом этапе:")
for step, count in funnel_steps.items():
    print(f"{step}: {count}")

print("\n📈 Конверсия между этапами:")
prev_count = None
for step, count in funnel_steps.items():
    if prev_count is not None:
        conversion = (count / prev_count) * 100
        print(f"{step}: {conversion:.1f}% (из предыдущего этапа)")
    else:
        print(f"{step}: 100% (базовый уровень)")
    prev_count = count

# Общая конверсия
overall_conversion = (funnel_steps['Checkout'] / funnel_steps['Главная страница']) * 100
print(f"\n🎯 Общая конверсия в checkout: {overall_conversion:.1f}%")

## ⏱️ Анализ времени на странице

In [ ]:
# Анализ времени на странице
print("⏱️  АНАЛИЗ ВРЕМЕНИ НА СТРАНИЦЕ")
print("=" * 50)

# Распределение времени сессий
print("📊 РАСПРЕДЕЛЕНИЕ ВРЕМЕНИ СЕССИЙ:")
time_stats = df_clean['session_time_minutes'].describe()
print(f"Мин: {time_stats['min']:.2f} мин")
print(f"25%: {time_stats['25%']:.2f} мин")
print(f"Медиана: {time_stats['50%']:.2f} мин")
print(f"75%: {time_stats['75%']:.2f} мин")
print(f"Макс: {time_stats['max']:.2f} мин")

# Время на страницу (приблизительно)
df_clean['time_per_page'] = df_clean['session_time_minutes'] / df_clean['pages_count']
df_clean['time_per_page'] = df_clean['time_per_page'].replace([np.inf, -np.inf], np.nan)

print(f"\n📄 СРЕДНЕЕ ВРЕМЯ НА СТРАНИЦУ: {df_clean['time_per_page'].mean():.2f} мин")
print(f"📄 МЕДИАННОЕ ВРЕМЯ НА СТРАНИЦУ: {df_clean['time_per_page'].median():.2f} мин")

## 💰 Анализ конверсии до Checkout

In [ ]:
# Детальный анализ конверсии
print("💰 ДЕТАЛЬНЫЙ АНАЛИЗ КОНВЕРСИИ ДО CHECKOUT")
print("=" * 60)

# Конверсия по источникам трафика
print("📊 КОНВЕРСИЯ ПО ИСТОЧНИКАМ ТРАФИКА:")
conversion_by_source = df_clean.groupby('Referrer').agg({
    'Display Name': 'count',
    'has_checkout': 'sum',
    'has_checkout': 'mean'
}).round(3)

conversion_by_source.columns = ['Всего сессий', 'Checkout сессии', 'Конверсия']
conversion_by_source['Конверсия %'] = (conversion_by_source['Конверсия'] * 100).round(1)
print(conversion_by_source)

# Конверсия по длительности сессии
print("\n⏱️  КОНВЕРСИЯ ПО ДЛИТЕЛЬНОСТИ СЕССИИ:")
conversion_by_time = df_clean.groupby('session_category').agg({
    'Display Name': 'count',
    'has_checkout': 'sum',
    'has_checkout': 'mean'
}).round(3)

conversion_by_time.columns = ['Всего сессий', 'Checkout сессии', 'Конверсия']
conversion_by_time['Конверсия %'] = (conversion_by_time['Конверсия'] * 100).round(1)
print(conversion_by_time)

## 📊 Визуализация данных

In [ ]:
# Создание графиков
print("📊 СОЗДАНИЕ ВИЗУАЛИЗАЦИИ")
print("=" * 50)

# 1. Распределение времени сессий
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.hist(df_clean['session_time_minutes'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('Распределение времени сессий')
plt.xlabel('Время (минуты)')
plt.ylabel('Количество сессий')
plt.grid(True, alpha=0.3)

# 2. Распределение количества страниц
plt.subplot(2, 2, 2)
page_counts = df_clean['pages_count'].value_counts().sort_index()
plt.bar(page_counts.index, page_counts.values, color='lightgreen', alpha=0.7)
plt.title('Распределение количества страниц')
plt.xlabel('Количество страниц')
plt.ylabel('Количество сессий')
plt.grid(True, alpha=0.3)

# 3. Конверсия по сегментам
plt.subplot(2, 2, 3)
conversion_data = df_clean.groupby('session_category')['has_checkout'].mean() * 100
plt.bar(conversion_data.index, conversion_data.values, color='gold', alpha=0.7)
plt.title('Конверсия по сегментам времени')
plt.xlabel('Сегмент')
plt.ylabel('Конверсия (%)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 4. Источники трафика
plt.subplot(2, 2, 4)
traffic_sources = df_clean['Referrer'].value_counts().head(5)
plt.pie(traffic_sources.values, labels=traffic_sources.index, autopct='%1.1f%%', startangle=90)
plt.title('Источники трафика')

plt.tight_layout()
plt.show()

print("✅ Графики созданы")

## 🔍 Инсайты и рекомендации

In [ ]:
# Анализ проблемных мест
print("🔍 АНАЛИЗ ПРОБЛЕМНЫХ МЕСТ")
print("=" * 50)

# 1. Страницы с высоким bounce rate
print("📊 СТРАНИЦЫ С ВЫСОКИМ BOUNCE RATE:")
starting_page_bounce = df_clean.groupby('Starting Page').agg({
    'Display Name': 'count',
    'pages_count': lambda x: (x == 1).sum()
})

starting_page_bounce['bounce_rate'] = (starting_page_bounce['pages_count'] / starting_page_bounce['Display Name'] * 100).round(1)
starting_page_bounce.columns = ['Всего сессий', 'Bounce сессии', 'Bounce Rate (%)']
starting_page_bounce = starting_page_bounce.sort_values('Bounce Rate (%)', ascending=False)

print(starting_page_bounce)

# 2. Анализ коротких сессий
print("\n⏱️  АНАЛИЗ КОРОТКИХ СЕССИЙ (< 1 минуты):")
short_sessions = df_clean[df_clean['session_time_minutes'] < 1]
print(f"Количество коротких сессий: {len(short_sessions)}")
print(f"Процент от общего числа: {(len(short_sessions)/len(df_clean)*100):.1f}%")

## 📈 Заключение и следующие шаги

In [ ]:
# Итоговый отчет
print("📈 ИТОГОВЫЙ ОТЧЕТ ПО АНАЛИЗУ")
print("=" * 50)

print(f"📊 ОБЩАЯ СТАТИСТИКА:")
print(f"- Всего сессий: {total_sessions}")
print(f"- Среднее время сессии: {avg_session_time:.2f} минут")
print(f"- Bounce rate: {bounce_rate:.1f}%")
print(f"- Конверсия в checkout: {checkout_conversion:.1f}%")

print(f"\n🎯 КЛЮЧЕВЫЕ ИНСАЙТЫ:")
print(f"- {len(df_clean[df_clean['session_time_minutes'] < 1])} сессий длились менее 1 минуты")
print(f"- {len(df_clean[df_clean['pages_count'] == 1])} пользователей покинули сайт после первой страницы")
print(f"- {len(df_clean[df_clean['has_checkout'] == True])} пользователей дошли до checkout")

print(f"\n🚀 ПРИОРИТЕТНЫЕ ДЕЙСТВИЯ:")
print("1. Оптимизация страниц с высоким bounce rate")
print("2. Улучшение checkout процесса")
print("3. A/B тестирование ключевых элементов")
print("4. Оптимизация скорости загрузки")